In [35]:
import numpy as np
import pandas as pd
import os
import urllib.request
import rpy2.robjects as robjects

In [36]:
# mat_cont: matriz quadrada (n x n) que será reagrupada
# aggregate_indices: lista de listas, cada sublista contém os índices das idades a serem agregadas.
  # Exemplo: [[0, 1], [2, 3]] agrupa as faixas etárias 0 e 1 em um único grupo e as faixas 2 e 3 em outro.
# age_structure (opcional): vetor que especifica a estrutura etária (tamanho relativo de cada grupo etário)
  #Por padrão, assume valores iguais (população uniformemente distribuída).
# silent (opcional): valor booleano
  # define se mensagens de aviso sobre discrepâncias em age_structure devem ser exibidas.
  # True (silencioso).

def aggregate_age_matrix(mat_cont, aggregate_indices, age_structure=None, silent=True):
    if age_structure is None:
        age_structure = np.ones(mat_cont.shape[0])

    age_structure = np.array(age_structure, dtype=float)
    #compara número de linhas com colunas
    if mat_cont.shape[0] != mat_cont.shape[1]:
        raise ValueError("mat_cont is not a square matrix")
    #compara número de faixas etárias
    if len(age_structure) < mat_cont.shape[0]:
        raise ValueError("age_structure is smaller than the linear dimension of mat_cont")

    if len(age_structure) > mat_cont.shape[0]:
        if silent == False:
            print("age_structure is bigger than linear size of mat_cont, aggregating the last values in one")
        #agrupa os últimos pesos
        age_structure[mat_cont.shape[0]-1] = sum(age_structure[mat_cont.shape[0]-1:])
    # matriz quadrada com o número de agrupamentos novos
    new_mat_cont = np.zeros((len(aggregate_indices), len(aggregate_indices)))

    for a, indices_i in enumerate(aggregate_indices):
        for b, indices_j in enumerate(aggregate_indices):
            Na = 0
            for i in indices_i:
                for j in indices_j:
                    new_mat_cont[a, b] += age_structure[i] * mat_cont[i, j]
                Na += age_structure[i]
            new_mat_cont[a, b] /= Na

    return new_mat_cont

In [31]:
# Estrutura etária
# Carrega a distribuição etária da população de Manaus em 2020
# age_structure = pd.read_csv('sample_data/Age_distribution_2020_Manaus.csv')

#Link direto para o arquivo CSV no GitHub (raw)
url1 = 'https://raw.githubusercontent.com/emanoelpereira/modelo-endemia/refs/heads/main/DATA/Age_distribution_2020_Manaus.csv'
url2 = 'https://raw.githubusercontent.com/emanoelpereira/modelo-endemia/refs/heads/main/DATA/BRA_data.csv'

# Carregar o arquivo CSV sem o cabeçalho
age_structure = pd.read_csv(url1)
mat_cont = pd.read_csv(url2)

# Se você quiser um vetor como uma lista (convertendo o DataFrame para uma lista)
age_structure = age_structure.values.flatten()  # Isso retorna um array 1D (vetor)

# Transformar em matriz (NumPy array)
mat_cont = mat_cont.to_numpy()  # Ou mat_cont.values

# Define os índices para agregação
aggregate_indices = [range(4), range(4, 8), range(8, 12), range(12, 16)]

matcontact_Manaus = aggregate_age_matrix(mat_cont, aggregate_indices, age_structure)

print (matcontact_Manaus)

[[10.11867896  3.78033088  2.70913848  0.9511593 ]
 [ 3.37764235  7.02985209  3.58734798  1.088056  ]
 [ 2.81234283  4.91393949  4.69436122  1.28141632]
 [ 1.83111481  2.96900897  2.79544812  1.75528531]]
